# Imports

In [ ]:
import sys
sys.path.append('..') 
from _utils import load_json, flatten_list # Adjust path to import from parent directory

In [ ]:
from tqdm import tqdm
import random
import pandas as pd

In [ ]:
from _helpers_parsing import extend_jobs

# Functions

In [ ]:
def choose_random_ads(df, k):
    random_choices = []
    ids_unique = df["esco_id"].unique()
    for id in tqdm(ids_unique):
        filtered_df = df[df["esco_id"] == id]
        if len(filtered_df) <= k:
            random_choices += filtered_df.to_dict("records")
        else:
            random_choices += random.sample(filtered_df.to_dict("records"), k)
    return random_choices

# Main

# load overview

In [ ]:
overview = pd.DataFrame(load_json("../00_data/EURES/eures_overview_total.json"))
print(len(overview))
overview = overview.drop_duplicates(["title", "url"])
print(len(overview))
overview = overview[~overview["title"].isnull()]
print(len(overview))

## add esco ids

In [ ]:
ads_extended = pd.DataFrame(flatten_list(overview.progress_apply(extend_jobs, axis=1)))
print(len(ads_extended))
ads_extended = ads_extended[~ads_extended["esco_id"].isnull()]
print(len(ads_extended))

In [ ]:
unique_ids_unparsed = list(ads_extended["esco_id"].unique())

## load parsed ads

In [ ]:
#exclude  already parsed ads in embedding dataset and deadlinks
parsed_ads = pd.DataFrame(load_json("../00_data/EURES/0_pars_short_ads_final"))
deadlinks = load_json("../00_data/EURES/deadlinks_final.json")

In [ ]:
parsed_urls = list(parsed_ads["url"].unique())

In [ ]:
print(len(ads_extended))
unparsed_ads_filtered = ads_extended[~ads_extended["url"].isin(parsed_urls)]
unparsed_ads_filtered = unparsed_ads_filtered[~unparsed_ads_filtered["url"].isin(deadlinks)]
len(unparsed_ads_filtered)

In [ ]:
len(unparsed_ads_filtered["esco_id"].unique())

In [ ]:
unparsed_ads_filtered["no_esco_jobs"] = unparsed_ads_filtered["esco_jobs"].apply(len)

## RESULTS

In [ ]:
import pandas as pd

In [ ]:
parsed_test_ads = pd.DataFrame(load_json("../00_data/EURES/eures_testads_final.json"))
len(parsed_test_ads)